In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os
import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random

In [164]:
SDB = sqlite3.connect('autoplius4.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists T1
(
gamintojas text,
modelis text,
kaina integer,
metai integer,
kebulas text,
kuras text,
pavaros text,
galingumas text,
rida integer,
nuorodos text,
pirmaregistracija text,
technikine text

)
'''
Cs.execute(sql)

sql_template = '''insert into T1 values (?,?,?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
for a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12 in zip(gamintojas, modelis, kaina, metai, kebulas, kuras, pavaros, galingumas, rida, nuorodos, pirmaregistracija, technikine):
    Cs.execute(sql_template, (a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12))

SDB.commit() # commit() būtinas, jei norite įrašyti duoenis į DB

SDB.close()

In [153]:
opcijos = Options()
opcijos.add_argument('--incognito')

modelis,gamintojas,metai,kebulas,kaina,kuras,pavaros,galingumas,rida,regsalis,nuorodos,pirmaregistracija=[],[],[],[],[],[],[],[],[],[],[],[]
technikine=[]


driver = webdriver.Chrome(options=opcijos)

for pgnr in range(1, 11):
    url = 'https://autoplius.lt/skelbimai/naudoti-automobiliai?page_nr='+str(pgnr) 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    automobiliai = bs.find_all('a', {'class':'announcement-item'})

    for auto in automobiliai:
        kuras.append(None)
        metai.append(None)
        rida.append(None)
        galingumas.append(None)
        kebulas.append(None)
        pavaros.append(None)
        technikine.append(None)
        pirmaregistracija.append(None)
        
        a=auto.find('div', {'class':'announcement-parameters-block'}).find_all('span')
        url1 = auto['href'] 
        driver.get(url1)
        time.sleep(2)
        source1 = driver.page_source
        bs1 = BeautifulSoup(source1, 'html.parser')
        
        for duomenys in bs1.find_all('div', {'class':'parameter-row'}):
            try:
                duomenys.find('div',{'class':'parameter-label'}).get_text().strip()
                duomenys.find('div',{'class':'parameter-value'}).get_text().strip()
            except:
                pass
            else:     
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Kuro tipas':
                    kuras[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Pirma registracija':
                    metai[-1]=int(duomenys.find('div',{'class':'parameter-value'}).get_text().strip()[:4])
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Rida':
                    rida[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip().replace(' km', '').replace(' ',''))
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Variklis':
                    galia=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip().split()[-1])
                    if str(galia[-3:-1]) == 'kW':
                        galingumas[-1]=((galia[:-3].replace('(', '')))
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Kėbulo tipas':
                    kebulas[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Pavarų dėžė':
                    pavaros[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Tech. apžiūra iki':
                    technikine[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Pirmosios registracijos šalis':
                    pirmaregistracija[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
        
        try:
            bs1.find_all('li', {'class':'crumb'})[2].get_text().strip()
        except:
            gamintojas.append(None)
            pass
        else:
            gamintojas.append(bs1.find_all('li', {'class':'crumb'})[2].get_text().strip())
            
        try:
            bs1.find_all('li', {'class':'crumb'})[3].get_text().strip()
        except:
            modelis.append(None)
            pass
        else:
            modelis.append(bs1.find_all('li', {'class':'crumb'})[3].get_text().strip())
            
        kaina.append(int(auto.find('div', {'class':'announcement-pricing-info'}).find('strong').get_text().strip().replace(' ','').replace('€', '')))
        nuorodos.append(auto['href'])
            
            
driver.close()

In [2]:
opcijos = Options()
opcijos.add_argument('--incognito')

driver = webdriver.Chrome(options=opcijos)
url = 'https://autoplius.lt/'
driver.get(url)
time.sleep(2)
source = driver.page_source
bs = BeautifulSoup(source, 'html.parser')

gamintojuid = bs.find('div', {'class':'form-row make-and-model-row'})
gamintojuid = gamintojuid.find_all('div', {'class':'dropdown-option js-option'})

gamintoju_id, gamintoju_kiekis=[], []
for gaminid in gamintojuid:
    gamintoju_id.append(gaminid['data-value'])
    gamintoju_kiekis.append(int(gaminid.find('div', {'class': 'badge'}).get_text()))
    
driver.close()


In [27]:
print(gamintoju_id)

# gamintoju_id=gamintoju_id[4:]
# gamintoju_kiekis=gamintoju_kiekis[4:]

['110', '90', '89', '29763', '87', '86', '85', '109', '41', '122', '34099', '84', '27783', '83', '82', '37152', '81', '80', '125', '79', '78', '33844', '77', '76', '75', '74', '73', '33999', '72', '18723', '71', '119', '33181', '33720', '30847', '69', '35741', '111', '68', '67', '66', '65', '16127', '64', '63', '116', '33872', '62', '60', '37164', '37216', '59', '58', '31465', '57', '56', '54', '53', '52', '51', '114', '49', '30799', '48', '47', '40', '46', '45', '120', '17577', '19524', '44', '124', '115', '15983', '43', '42', '118', '113', '36061', '37251']


In [28]:
def renkaduomenis(nr, kiek):
    opcijos = Options()
    opcijos.add_argument('--incognito')

    modelis,gamintojas,metai,kebulas,kaina,kuras,pavaros,galingumas,rida,nuorodos,pirmaregistracija=[],[],[],[],[],[],[],[],[],[],[]
    technikine=[]


    driver = webdriver.Chrome(options=opcijos)
    kiek= int(np.ceil(kiek/20))
    if kiek > 20:
        kiek = 20

    for pgnr in range(1, kiek+1):
        
        url = f'https://autoplius.lt/skelbimai/naudoti-automobiliai?make_id={nr}&page_nr='+str(pgnr) 
        try:
            
            skaiciukas = random.randint(2, 5)
            driver.get(url)
            time.sleep(skaiciukas)
            source = driver.page_source
            bs = BeautifulSoup(source, 'html.parser')
            automobiliai = bs.find_all('a', {'class':'announcement-item'})
        except:
            break
       
        for auto in automobiliai:
            kuras.append(None)
            metai.append(None)
            rida.append(None)
            galingumas.append(None)
            kebulas.append(None)
            pavaros.append(None)
            technikine.append(None)
            pirmaregistracija.append(None)
        
            # a=auto.find('div', {'class':'announcement-parameters-block'}).find_all('span')
            url1 = auto['href'] 
            driver.get(url1)
            time.sleep(2)
            source1 = driver.page_source
            bs1 = BeautifulSoup(source1, 'html.parser')
        
            for duomenys in bs1.find_all('div', {'class':'parameter-row'}):
                try:
                    duomenys.find('div',{'class':'parameter-label'}).get_text().strip()
                    duomenys.find('div',{'class':'parameter-value'}).get_text().strip()
                except:
                    pass
                else:     
                    if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Kuro tipas':
                        kuras[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                    if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Pirma registracija':
                        metai[-1]=int(duomenys.find('div',{'class':'parameter-value'}).get_text().strip()[:4])
                    if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Rida':
                        rida[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip().replace(' km', '').replace(' ',''))
                    if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Variklis':
                        galia=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip().split()[-1])
                        if str(galia[-3:-1]) == 'kW':
                            galingumas[-1]=((galia[:-3].replace('(', '')))
                    if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Kėbulo tipas':
                        kebulas[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                    if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Pavarų dėžė':
                        pavaros[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                    if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Tech. apžiūra iki':
                        technikine[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                    if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Pirmosios registracijos šalis':
                        pirmaregistracija[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
        
            try:
                bs1.find_all('li', {'class':'crumb'})[2].get_text().strip()
            except:
                gamintojas.append(None)
                pass
            else:
                gamintojas.append(bs1.find_all('li', {'class':'crumb'})[2].get_text().strip())
            
            try:
                bs1.find_all('li', {'class':'crumb'})[3].get_text().strip()
            except:
                modelis.append(None)
                pass
            else:
                modelis.append(bs1.find_all('li', {'class':'crumb'})[3].get_text().strip())
            try:
                kaina.append(int(auto.find('div', {'class':'announcement-pricing-info'}).find('strong').get_text().strip().replace(' ','').replace('€', '')))
            except:
                kaina.append(None)
            nuorodos.append(auto['href'])
            
            
    driver.close()
    return modelis,gamintojas,metai,kebulas,kaina,kuras,pavaros,galingumas,rida,nuorodos,pirmaregistracija,technikine






for i, j in zip(gamintoju_id, gamintoju_kiekis):
    modelis1,gamintojas1,metai1,kebulas1,kaina1,kuras1,pavaros1,galingumas1,rida1,nuorodos1,pirmaregistracija1=[],[],[],[],[],[],[],[],[],[],[]
    technikine1=[]
    
    modelis1,gamintojas1,metai1,kebulas1,kaina1,kuras1,pavaros1,galingumas1,rida1,nuorodos1,pirmaregistracija1,technikine1 = renkaduomenis(i, j)
    
    SDB = sqlite3.connect('autopliusvisi.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
    Cs = SDB.cursor()

    sql = '''create table if not exists T1
    (
    gamintojas text,
    modelis text,
    kaina integer,
    metai integer,
    kebulas text,
    kuras text,
    pavaros text,
    galingumas text,
    rida integer,
    nuorodos text,
    pirmaregistracija text,
    technikine text

    )
    '''
    Cs.execute(sql)

    sql_template = '''insert into T1 values (?,?,?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
    for a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12 in zip(gamintojas1, modelis1, kaina1, metai1, kebulas1, kuras1, pavaros1, galingumas1, rida1, nuorodos1, pirmaregistracija1, technikine1):
        Cs.execute(sql_template, (a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12))

    SDB.commit() # commit() būtinas, jei norite įrašyti duoenis į DB

    SDB.close()
    

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=129.0.6668.101)
Stacktrace:
	GetHandleVerifier [0x00007FF6F342B095+29557]
	(No symbol) [0x00007FF6F339FA50]
	(No symbol) [0x00007FF6F325B56A]
	(No symbol) [0x00007FF6F3241528]
	(No symbol) [0x00007FF6F32425BF]
	(No symbol) [0x00007FF6F325BB83]
	(No symbol) [0x00007FF6F322F7AD]
	(No symbol) [0x00007FF6F32E6339]
	(No symbol) [0x00007FF6F32D7150]
	(No symbol) [0x00007FF6F32A116F]
	(No symbol) [0x00007FF6F32A22D1]
	GetHandleVerifier [0x00007FF6F375C96D+3378253]
	GetHandleVerifier [0x00007FF6F37A8497+3688311]
	GetHandleVerifier [0x00007FF6F379D1CB+3642539]
	GetHandleVerifier [0x00007FF6F34EA6B6+813462]
	(No symbol) [0x00007FF6F33AAB5F]
	(No symbol) [0x00007FF6F33A6B74]
	(No symbol) [0x00007FF6F33A6D10]
	(No symbol) [0x00007FF6F3395C1F]
	BaseThreadInitThunk [0x00007FFB06CC257D+29]
	RtlUserThreadStart [0x00007FFB07F4AF08

In [31]:
print(a1)

None
